## tool: KNNFeatureAggregator (4 балла)

Нужно написать класс, который будет справляться с задачей генерации новых фичей по ближайшим соседям.
Принцип его работы объясним на примере. Допустим, мы находимся в каком-то пайплайне генерации признаков. Разберем псевдокод ниже:
```python
# 1
'''
    Создаем объект нашего класса - он принимает на вход информацию о том, какой будет индекс для поиска ближ. соседей.
    Далее, "обучаем" индекс, если это нужно делать (строим граф, строим ivf-табличку) и т.п.).
    После этого блока, у нас есть обученный индекс, готовый искать ближайших соседей по train_data.
'''
knn_feature_aggregator = KNNFeatureAggregator(index_info)
knn_feature_aggregator.train(train_data, index_add_info)

# 2
'''
    Считаем индексы ближайших соседей. На данном этапе мы хотим получить признаки для обучающей выборки, поэтому
        подаем в качестве query_data нашу обучалку.
    Указывам is_train=True, чтобы вернуть k ближайших соседей без учета самих себя (считая k+1 соседей + выкидывая 1 столбик).
    k указываем __МАКСИМАЛЬНОЕ_ИЗ_ТРЕБУЮЩИХСЯ_НИЖЕ__ (пока не анализируем что это значит, просто имеем в виду).

    Возвращает np.array размера (query_data.shape[0], k) с айдишниками ближ. соседей
'''
train_neighbors = knn_feature_aggregator.kneighbors(
        query_data=train_data,
        k=100,
        is_train=True,
        index_add_info=index_add_info
)

# 4 (сначала см. пункт 3 ниже)
'''
    Информацию о признаках можно подавать, например, в виде такого словаря.
    Ключи - названия результирующих колонок с новыми признаками.
    Значения - таплы из:
        1. Название оригинальной колонки, по которой агрегируемся
        2. Аггрегирующая фукнция
        3. Список из количества ближайших соседей, по которым считаем агг. функцию.
            Здесь каждое число должно быть НЕ БОЛЬШЕ k из пункта 2 (вспоминаем "__МАКСИМАЛЬНОЕ_ИЗ_ТРЕБУЮЩИХСЯ_НИЖЕ__", понимаем :)

    Пример:
        Имеем из п. 2 айдишники соседей:
        train_neighbors = array([[1, 2, 3],
                                 [2, 0, 3],
                                 [3, 1, 4],
                                 [4, 2, 1],
                                 [3, 2, 1]], dtype=uint64)

        Тогда по записи {
            ...
            'new_neighbors_age_mean': ('age', 'mean', [2, 3]),
        }

        Создадутся две новых колонки - 'new_neighbors_age_mean_2nn', 'new_neighbors_age_mean_3nn'.
        В первой будет для каждого объекта лежать средний возраст его двух ближ. соседей,
            во второй - средний возраст трех ближ. соседей.

'''
feature_info =
{
                    #  название_колонки     агг.функция               список кол-ва соседей, по которым считать агг. функцию
    'new_col_name_1': ('original_col_name_1',     'sum',                                [10, 20, 100]),
    'new_col_name_2': ('original_col_name_2',     lambda x: x.min() % 3,                [50, 80, 100])
}

# 3
'''
    Суть этого класса - генерировать новые фичи на основе ближайших соседей. Здесь мы это и делаем.
    Для этого подаем на вход айдишники соседей из обучающей выборки и саму обучающую выборку.
    Далее, подаем на вход информацию о том, "какие" признаки нам нужны, см. выше.

    Возвращает датафрейм размера (neighbor_ids.shape[0], количество_новых_фичей_по_feature_info)
'''
train_new_feature_df = knn_feature_aggregator.make_features(
    neighbor_ids=train_neighbors,
    train_data=train_data,
    feature_info=feature_info
)
train_data_with_new_features = merge(train_data, train_new_feature_df)

# 5
'''
    Для тестовой выборки пайплайн будет выглядеть аналогично, за исключением того, что is_train теперь False
'''
test_neighbors = knn_feature_aggregator.kneighbors(
        query_data=test_data,
        k=100,
        is_train=False,
        index_add_info=index_add_info
)
test_new_feature_df = knn_feature_aggregator.make_features(
    neighbor_ids=test_neighbors,
    train_data=train_data,
    feature_info=feature_info
)
test_data_with_new_features = merge(test_data, test_new_feature_df)

```

### Задание:
Написать класс, который реализует все, что описано выше, в частности:

**\_\_init\_\_**
- вы сами решаете, какой будет индекс, будет ли он фиксирован и т.п.

**train**
- обучающую выборку не нужно сохранять в объект класса в целях экономии памяти
- если вам нужно разбить `train` на `train` и `add_items`,
      чтобы поддерживать обучение индекса на репрезентативном сабсэмпле, можете это сделать
- аргумент train_data - не обязательно выборка со всеми признаками.
      Вы хотите подавать сюда то подмножество признаков, по которому будете искать соседей
      (соответственно, нужно подавать уже приведенные к однородному виду данные)

**kneighbors**
- обязательна поддержка флажка is_train с описанным выше функционалом
- аргумент query_data - см. замечание к аргументу train_data из метода train выше

**make_features**
- обработайте отдельно случай, когда вы в качестве ближайших соседей подаете единственное число.
      Не нужно извне подавать список из одного числа, обработка должна быть внутри

**Эффективность**

Все должно быть реализовано эффективно. В том числе:
- без цикла for по всем объектам train_data/query_data
- без pd.DataFrame.apply
- можно использовать np.apply_along_axis (работает в ~5 раз быстрее, чем pandas)

**Пример**

Нужно привести пример работы вашего класса, запустив ячейки в блоке "Пример" ниже.
Не удаляйте авторский пример!

**Вопросы**

Нужно ответить на вопросы в блоке "Вопросы" ниже

**Note:** feature_info можете реализовать в любом виде, но описанный выше способ хорош тем,
      что его легко привести в удобный для дальнейшей работы вид:

In [1]:
import pandas as pd

feature_info = {
                    #  название_колонки     агг.функция               список кол-ва соседей, по которым считать агг. функцию
    'new_col_name_1': ('original_col_name_1',     lambda x: x.sum(),                                [10, 20, 100]),
    'new_col_name_2': ('original_col_name_1',     lambda x: x.mean(),                                [11, 21, 101]),
    'new_col_name_3': ('original_col_name_2',     lambda x: x.min() % 3,                [50, 80, 100])
}
pd.DataFrame(feature_info, index=['col_name', 'func', 'k']).T.explode('k').reset_index(names='new_col')

,new_col,col_name,func,k
0,new_col_name_1,original_col_name_1,<function <lambda> at 0x0000020E61B6FC40>,10
1,new_col_name_1,original_col_name_1,<function <lambda> at 0x0000020E61B6FC40>,20
2,new_col_name_1,original_col_name_1,<function <lambda> at 0x0000020E61B6FC40>,100
3,new_col_name_2,original_col_name_1,<function <lambda> at 0x0000020E616AB880>,11
4,new_col_name_2,original_col_name_1,<function <lambda> at 0x0000020E616AB880>,21
5,new_col_name_2,original_col_name_1,<function <lambda> at 0x0000020E616AB880>,101
6,new_col_name_3,original_col_name_2,<function <lambda> at 0x0000020E61A93B00>,50
7,new_col_name_3,original_col_name_2,<function <lambda> at 0x0000020E61A93B00>,80
8,new_col_name_3,original_col_name_2,<function <lambda> at 0x0000020E61A93B00>,100


In [2]:
import pynndescent
import numpy as np

In [3]:
class KNNFeatureAggregator:
    def __init__(self):  
        pass

    def train(self, train_data, **index_params):
        ''' 
            Обучение индекса PyNNDescent

            train_data -- на каких данных нужно строить индекс
            index_params -- параметры индекса из библиотеки PyNNDescent, такие как
            n_trees, n_neighbors, metric и т.д.
        '''
        self.index = pynndescent.NNDescent(data=train_data, **index_params)
        self.index.prepare()


    def kneighbors(self, query_data, is_train, k):
        ''' 
            Находит k ближайших соседей к объектам из query_data и возвращает
            tuple из этих соседей и расстояний до них

            is_train -- если True, то самый ближайший сосед игнорируется
        '''
        neighbors, _ = self.index.query(query_data, k=k+is_train)
        return neighbors[:,is_train:]

    def make_features(self, neighbor_ids, train_data, feature_info):
        ''' 
            Создаёт новые признаки с помощью аггрегации по ближайшим соседям 

            neighbor_ids -- индексы ближайших соседей в обучающей выборке
            train_data -- обучающая выборка
            feature_info -- словарь из таплов, содержащий информацию о требуемых новых фичах
            Ключи - названия результирующих колонок с новыми признаками.
            Значения - таплы из:
                1. Название оригинальной колонки, по которой агрегируемся
                2. Аггрегирующая фукнция
                3. Список из количества ближайших соседей, по которым считаем агг. функцию.
        '''
        new_features = pd.DataFrame()
        for key in feature_info:
            column_name, agg_func, k_neighbors = feature_info[key]
            if isinstance(k_neighbors, int):
                k_neighbors = [k_neighbors]
            for k in k_neighbors:
                feature_name = key + f'_{k}nn'
                column = train_data[column_name].to_numpy()
                feature = np.apply_along_axis(agg_func, axis=1, arr=column[neighbor_ids[:,:k]])
                feature = pd.DataFrame(feature, columns=[feature_name])
                new_features = pd.concat((new_features, feature), axis=1)
        return new_features

### Пример

#### Ваш:

Пример 1 (`is_train=True`):

In [4]:
train_data = pd.DataFrame({
    'a': [1, 2, 3, 4, 5],
    'b': [10, 19, 27, 34, 40]
})
agg = KNNFeatureAggregator()
agg.train(train_data, metric='euclidean', n_neighbors=3)
neighbor_ids = agg.kneighbors(train_data, is_train=True, k=3)
neighbor_ids

array([[1, 2, 3],
       [2, 0, 3],
       [3, 1, 4],
       [4, 2, 1],
       [3, 2, 1]])

In [5]:
X = agg.make_features(neighbor_ids, train_data, feature_info={
    'a_sum': ('a', lambda x: x.sum(), [2, 3]),
    'b_whatever': ('b', lambda x: x.min(), 4),
})
X

,a_sum_2nn,a_sum_3nn,b_whatever_4nn
0,5,9,19
1,4,8,10
2,6,11,19
3,8,10,19
4,7,9,19


Пример 2 (`is_train = False`):

In [6]:
test_data = pd.DataFrame({
    'a': [5, 4, 3, 4, 1],
    'b': [14, 11, 23, 34, 60]
})

neighbor_ids = agg.kneighbors(test_data, is_train=False, k=3)
neighbor_ids

array([[0, 1, 2],
       [0, 1, 2],
       [2, 1, 3],
       [3, 4, 2],
       [4, 3, 2]])

In [7]:
X = agg.make_features(neighbor_ids, train_data, feature_info={
    'a_sum': ('a', lambda x: x.sum(), [2, 3]),
    'b_whatever': ('b', lambda x: x.min(), 4),
})
X

,a_sum_2nn,a_sum_3nn,b_whatever_4nn
0,3,6,10
1,3,6,10
2,5,9,19
3,9,12,27
4,9,12,27


#### Авторский:

In [20]:
train_data = pd.DataFrame({
    'a': [1, 2, 3, 4, 5],
    'b': [10, 19, 27, 34, 40]
})
agg = KNNFeatureAgg(dim=2, metric='l2') # у автора: hnsw index
agg.train(train_data)
neighbor_ids = agg.kneighbors(train_data, is_train=True, k=3)
neighbor_ids # у вас индексы ближ. соседей могут отличаться

array([[1, 2, 3],
       [2, 0, 3],
       [3, 1, 4],
       [4, 2, 1],
       [3, 2, 1]], dtype=uint64)

In [21]:
X = agg.make_features(neighbor_ids, feature_info={
    'a_sum': ('a', lambda x: x.sum(), [2, 3]),
    'b_whatever': ('b', lambda x: x.min(), 2),
})
X

,a_sum_2nn,b_whatever_2nn,a_sum_3nn
0,5,19,9
1,4,10,8
2,6,19,11
3,8,27,10
4,7,27,9


### Вопросы

1) Какой / какие индекс[-ы] вы решили использовать для этой задачи и почему?
2) Какие недостатки / потенциальные зоны для улучшения у вашей текущей реализации?

### Ответы
1. Я решил использовать индекс из библиотеки `PyNNDescent`, потому что на мой взгляд это самый универсальный индекс (к тому же, там можно использовать кастомную метрику, что очень сильно подкупает).
2. Можно в будущем добавить поддержку индексов из других библиотек (а также поддержку кастомных индексов со стандартизированным интерфейсом)